In [1]:
import psycopg
import pandas as pd

db_connection = "dbname=malaria user=sepmein"
adm0_rainfall_path = '~/x/snt/inst/extdata/rainfall_adm0.csv'
adm1_rainfall_path = '~/x/snt/inst/extdata/rainfall_adm1.csv'
adm2_rainfall_path = '~/x/snt/inst/extdata/rainfall_adm2.csv'

In [14]:
rainfall_adm0 = pd.read_csv(adm0_rainfall_path)
adm0_tuples = [tuple(x) for x in rainfall_adm0.to_numpy()]

rainfall_adm1 = pd.read_csv(adm1_rainfall_path)
adm1_tuples = [tuple(x) for x in rainfall_adm1.to_numpy()]

rainfall_adm2 = pd.read_csv(adm2_rainfall_path)
rainfall_adm2 = rainfall_adm2.reindex(["GUID", "year", "month", "indicator", "value"], axis = 1)
adm2_tuples = [tuple(x) for x in rainfall_adm2.to_numpy()]

In [15]:
rainfall_adm2

,GUID,year,month,indicator,value
0,6fd639bc-8f0f-4c9a-b87e-a2a99628dc7b,2014,1,rainfall,454.319282
1,be3929b2-f0a7-4de7-af41-8c3c3145610e,2014,1,rainfall,362.782434
2,f6312c2e-9fed-446e-9b9d-10af8a41f630,2014,1,rainfall,365.283841
3,ad71adde-f81e-486f-97ec-3b691615f23b,2014,1,rainfall,402.067876
4,33b69006-b7b1-4f01-b448-c3f693a80ec4,2014,1,rainfall,297.677247
...,...,...,...,...,...
4604470,9a9000cc-a291-4b83-9b4b-6f32b629f57d,2020,10,rainfall,442.553371
4604471,719b85d0-167c-4a40-8254-3234d65b530e,2020,10,rainfall,460.669281
4604472,23edf0b3-8151-4453-8db2-c376d46a9adc,2020,10,rainfall,427.077774
4604473,947867cd-1112-42de-bd47-d1a3fd22d5f4,2020,10,rainfall,114.340191


In [16]:
sql_insert_adm0_monthly = """
insert into malaria
                    (adm0_id, ts_month_id, indicator_id, value)
                    WITH
                      g AS (
                        SELECT gid
                        FROM adm0
                        WHERE guid=%s
                      ),
                      t AS (
                        SELECT id
                        FROM ts_month
                        WHERE year=%s AND month=%s
                      ),
                      ind as (
  	                    select id
	                      from indicator
	                      where name=%s
                      )
                      select g.gid, t.id, ind.id, %s
                      from g, t, ind;
"""

sql_insert_adm1_monthly = """
insert into malaria
                    (adm1_id, ts_month_id, indicator_id, value)
                    WITH
                      g AS (
                        SELECT gid
                        FROM adm1
                        WHERE guid=%s
                      ),
                      t AS (
                        SELECT id
                        FROM ts_month
                        WHERE year=%s AND month=%s
                      ),
                      ind as (
  	                    select id
	                      from indicator
	                      where name=%s
                      )
                      select g.gid, t.id, ind.id, %s
                      from g, t, ind;
"""

sql_insert_adm2_monthly = """
insert into malaria
                    (adm2_id, ts_month_id, indicator_id, value)
                    WITH
                      g AS (
                        SELECT gid
                        FROM adm2
                        WHERE guid=%s
                      ),
                      t AS (
                        SELECT id
                        FROM ts_month
                        WHERE year=%s AND month=%s
                      ),
                      ind as (
  	                    select id
	                      from indicator
	                      where name=%s
                      )
                      select g.gid, t.id, ind.id, %s
                      from g, t, ind;
"""

In [ ]:
# Connect to an existing database
with psycopg.connect(db_connection) as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Pass data to fill a query placeholders and let Psycopg perform
        # the correct conversion (no SQL injections!)
        # cur.executemany(sql_insert_adm0_monthly, adm0_tuples)

        cur.executemany(sql_insert_adm1_monthly, adm1_tuples)
        conn.commit()
        
        cur.executemany(sql_insert_adm2_monthly, adm2_tuples)

        # Make the changes to the database persistent
        conn.commit()